## Initialize Terraform

In [1]:
#rm -rf .terraform rm .terraform.lock.hcl terraform.tfstate terraform.tfstate.backup
!terraform init


Initializing the backend...

Initializing provider plugins...
- Finding hashicorp/azurerm versions matching "3.90.0"...
- Installing hashicorp/azurerm v3.90.0...
- Installed hashicorp/azurerm v3.90.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


## View `resource-group.tf`

In [2]:
!type resource-group.tf #use this on WIndows
!cat resource-group.tf

# Creates a resource group in your Azure account.

resource "azurerm_resource_group" "main" {
  name     = var.app_name
  location = var.location
}


## View `container-registry.tf`

In [3]:
!type container-registry.tf #use this on Windows
!cat container-registry.tf

# Creates a container registry in Azure (for Docker images).
# Note!
# - Resource "azurerm_resource_group.main" with a property "name" is defined in the file "resource-group.tf".
# - The value for "resource_group_name" below is set using property "name" in resource "azurerm_resource_group.main":
#   - resource_group_name = azurerm_resource_group.main.name
# - "name" and "location" below are set from Terraform variables defined in the file "variables.tf".


resource "azurerm_container_registry" "main" {
  name                = var.app_name
  resource_group_name = azurerm_resource_group.main.name
  location            = var.location
  admin_enabled       = true
  sku                 = "Basic"
}


## Terraform Apply

In [4]:
!terraform apply -auto-approve


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # azurerm_container_registry.main will be created
  + resource "azurerm_container_registry" "main" {
      + admin_enabled                 = true
      + admin_password                = (sensitive value)
      + admin_username                = (known after apply)
      + encryption                    = (known after apply)
      + export_policy_enabled         = true
      + id                            = (known after apply)
      + location                      = "westeurope"
      + login_server                  = (known after apply)
      + name                          = "tsfn14g00"
      + network_rule_bypass_option    = "AzureServices"
      + network_rule_set              = (known after apply)
      + public_network_access_enabled = true
      + resource_group_name           = "tsf

## List Azure Resource Groups

In [5]:
!az group list -o table

Name       Location    Status
---------  ----------  ---------
tsfn14g00  westeurope  Succeeded


## List Azure Container Registries

In [6]:
!az acr list -o table

NAME       RESOURCE GROUP    LOCATION    SKU    LOGIN SERVER          CREATION DATE         ADMIN ENABLED
---------  ----------------  ----------  -----  --------------------  --------------------  ---------------
tsfn14g00  tsfn14g00         westeurope  Basic  tsfn14g00.azurecr.io  2024-02-02T17:10:55Z  True


## List Images in Container Registry `tsfn14g00 `

In [7]:
!az acr repository list -n tsfn14g00 --top 10 -o table

## Show Information about Azure Container Registry `tsfn14g00`

- You can use this command to get the `LOGIN SERVER` which is the URL to your Container Registry on Azure.
  - This is the URL you would use to upload Docker Containers to the Azure Container Registry.

In [8]:
!az acr show -n tsfn14g00 -o table

NAME       RESOURCE GROUP    LOCATION    SKU    LOGIN SERVER          CREATION DATE         ADMIN ENABLED
---------  ----------------  ----------  -----  --------------------  --------------------  ---------------
tsfn14g00  tsfn14g00         westeurope  Basic  tsfn14g00.azurecr.io  2024-02-02T17:10:55Z  True

CONTAINER_REGISTRY_LOGIN_SERVER: tsfn14g00.azurecr.io


## Show Credentials for Azure Container Registry `tsfn14g00`

- You can use this command to get the `USERBAME` and `PASSWORD` to use to authenticate with your Azure Container Registry.

In [9]:
!az acr credential show -n tsfn14g00 -o table

USERNAME    PASSWORD                                              PASSWORD2
----------  ----------------------------------------------------  ----------------------------------------------------
tsfn14g00   IqzyfIYZTRJOYFe+amTGQGdomW5nPfgqZMLon6MG2Z+ACRAMjKNe  yhL5NskBFWSUHMzyNmgy6pefO6qSxfnI8vsEuGTQfZ+ACRB8Y8bw

CONTAINER_REGISTRY_USERNAME: tsfn14g00
CONTAINER_REGISTRY_PASSWORD: IqzyfIYZTRJOYFe+amTGQGdomW5nPfgqZMLon6MG2Z+ACRAMjKNe


## Login to Azure Container Registry via Docker

- Replace the variables below with your `LOGIN_SERVER`, `USERNAME` and `PASSWORD`.

In [10]:
!echo $PASSWORD | docker login $LOGIN_SERVER -u $USERNAME --password-stdin  > /dev/null 2>&1

## Build and Push Docker Image to Azure Container Registry

- Note, this code has only been tested on Ubuntu.

In [11]:
# Prepare Nodejs Application
#rm -rf ./video-streaming/node_modules ./video-streaming/package.json ./video-streaming/package-lock.json
cd ./video-streaming && \
npm init -y && \
npm pkg set 'main'='./src/index.js' && \
npm pkg set 'scripts.start'='node ./src/index.js' && \
npm pkg set 'scripts.start:dev'='nodemon ./src/index.js' && \
npm pkg delete 'scripts.test' && \
npm install --silent --save express@5.0.0-beta.1 && \
npm install --silent --save-dev nodemon@3.0.3 && \
cd ..

# Build Docker image with Nodejs Application
docker build -q -t video-streaming:1 -f ./video-streaming/Dockerfile ./video-streaming
docker tag video-streaming:1 $CONTAINER_REGISTRY_LOGIN_SERVER/video-streaming:1

# Push Docker Image to Azure Container Registry
docker push $CONTAINER_REGISTRY_LOGIN_SERVER/video-streaming:1

# Clean up
docker rmi video-streaming:1 > /dev/null 2>&1
docker rmi $CONTAINER_REGISTRY_LOGIN_SERVER/video-streaming:1 > /dev/null 2>&1
docker image list | grep -i -E 'REPOSITORY|video-streaming'

Wrote to /home/patrick/projects/tsfn14/03_container_registry/video-streaming/package.json:

{
  "name": "video-streaming",
  "version": "1.0.0",
  "description": "",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC"
}


(##################) ⠹ reify:nodemon: timing reifyNode:node_modules/semver Compulm[?25h(⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂) ⠦ idealTree:video-streaming: sill idealTree buildDeps
failed to fetch metadata: fork/exec /usr/local/lib/docker/cli-plugins/docker-buildx: no such file or directory

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

sha256:3ca223f84f9be92120d070261a7e96ef36ecc0d74f5aa2d36c5e50bd49003708
The push refers to repository [tsfn14g00.azurecr.io/video-streaming]

7c64522f: Preparing 
1416c0e3: Preparing 
5ef28cc8: Pre

## List Images in Container Registry `tsfn14g00 `

In [12]:
!az acr repository list -n tsfn14g00 --top 10 -o table

Result
---------------
video-streaming


## Show Info. about Image `video-streaming` in Azure Container Registry

In [13]:
!az acr repository show -n tsfn14g00 --image video-streaming:1 -o table
!echo ''
!az acr repository show -n tsfn14g00 --repository video-streaming -o table
!echo ''
!az acr repository show-tags -n tsfn14g00 --repository video-streaming --top 10 -o table

CreatedTime                  Digest                                                                   LastUpdateTime               Name    Signed
---------------------------  -----------------------------------------------------------------------  ---------------------------  ------  --------
2024-02-02T17:12:25.800828Z  sha256:61bd97058495742153b841265eeeb39204ba70b02a23975fddb1847976cb9fff  2024-02-02T17:12:25.800828Z  latest  False

CreatedTime                   ImageName        LastUpdateTime                ManifestCount    Registry              TagCount
----------------------------  ---------------  ----------------------------  ---------------  --------------------  ----------
2024-02-02T17:12:25.6611897Z  video-streaming  2024-02-02T17:12:25.7665149Z  1                tsfn14g00.azurecr.io  1

Result
--------
latest


## Show Azure Container Registry Usage

In [14]:
!az acr show-usage -n tsfn14g00 -o table

NAME       LIMIT        CURRENT VALUE    UNIT
---------  -----------  ---------------  ------
Size       10737418240  56542045         Bytes
Webhooks   2            0                Count
ScopeMaps  100          0                Count
Tokens     100          0                Count


## Pull and Run Docker Image from Azure Container Registry

In [15]:
!docker run --name video-streaming  -d -p 3000:3000 -e PORT=3000 $CONTAINER_REGISTRY_LOGIN_SERVER/video-streaming:1
!docker ps
!docker logs video-streaming
#!docker exec -it video-streaming /bin/sh

## Open URL `http://localhost:3000/video` in Firefox browser
!firefox http://localhost:3000/video

Unable to find image 'tsfn14g00.azurecr.io/video-streaming:latest' locally
latest: Pulling from video-streaming

fdb3b6a5: Already exists 
750296b3: Already exists 
bc07f19b: Already exists 
83a79857: Already exists 
b12b63f4: Pulling fs layer 
f42ac151: Pulling fs layer 
0b73d6d3: Pulling fs layer 
3de62452: Pulling fs layer 
Digest: sha256:61bd97058495742153b841265eeeb39204ba70b02a23975fddb1847976cb9fff
Status: Downloaded newer image for tsfn14g00.azurecr.io/video-streaming:latest
e0aedca03cd0d48eaeb5c15a777efd5529272c0143f5706771ccecb0edb0b0eb
CONTAINER ID   IMAGE                                         COMMAND                  CREATED        STATUS                  PORTS                                       NAMES
e0aedca03cd0   tsfn14g00.azurecr.io/video-streaming:latest   "docker-entrypoint.s…"   1 second ago   Up Less than a second   0.0.0.0:3000->3000/tcp, :::3000->3000/tcp   video-streaming
[GFX1-]: glxtest: ManageChildProcess failed



## Stop and Remove Container and Local Image

In [16]:
!docker stop video-streaming
!docker rm video-streaming
!docker ps -a | grep -i -E 'IMAGE|video-streaming'
!docker rmi $CONTAINER_REGISTRY_LOGIN_SERVER/video-streaming:1 > /dev/null 2>&1
!docker image list | grep -i -E 'REPOSITORY|video-streaming'

video-streaming
video-streaming
CONTAINER ID   IMAGE                                 COMMAND                  CREATED      STATUS                    PORTS                                                                                                                                  NAMES
REPOSITORY                    TAG             IMAGE ID       CREATED         SIZE


## Logout from Azure Container Registry via Docker

In [17]:
!docker logout $CONTAINER_REGISTRY_LOGIN_SERVER > /dev/null 2>&1

## Delete Image `video-streaming` from Azure Container Registry

In [18]:
#!az acr repository delete -y -n tsfn14g00 --image video-streaming:1 -table
!az acr repository delete -y -n tsfn14g00 --repository video-streaming -o table

## List Images in Container Registry `tsfn14g00 `

In [19]:
!az acr repository list -n tsfn14g00 --top 10 -o table

## Terraform Destroy

In [20]:
!terraform destroy -auto-approve

azurerm_resource_group.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00]
azurerm_container_registry.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00/providers/Microsoft.ContainerRegistry/registries/tsfn14g00]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # azurerm_container_registry.main will be destroyed
  - resource "azurerm_container_registry" "main" {
      - admin_enabled                 = true -> null
      - admin_password                = (sensitive value) -> null
      - admin_username                = "tsfn14g00" -> null
      - anonymous_pull_enabled        = false -> null
      - data_endpoint_enabled         = false -> null
      - encryption                    = [
          - {
              - enabled    

## List Azure Container Registries

In [21]:
!az acr list -o table

## List Azure Resource Groups

In [22]:
!az group list -o table